In [1]:
import xml.dom.minidom
import xml.etree.ElementTree as ET
from xml.dom.minidom import Node
import pandas as pd

In [47]:
Corrected_Numbers = pd.read_csv("2021_Q1-Q2 TEC Ambulatory Pt Volume.csv")
Q1 = Corrected_Numbers[Corrected_Numbers.Month.isin([1,2,3])]
Q2 = Corrected_Numbers[Corrected_Numbers.Month.isin([4,5,6])]
Q3 = Corrected_Numbers[Corrected_Numbers.Month.isin([7,8,9])]
Q4 = Corrected_Numbers[Corrected_Numbers.Month.isin([10,11,12])]

In [2]:
Amb_Patient_Volume = pd.read_csv("2021_Q2 Ambulatory Pt Volume.csv")

In [3]:
Amb_Patient_Volume

,Month,Hospital,UnitName,TotalNumberofPatientVisits
0,4,EUH,EUH EP Lab,273
1,5,EUHM,EUHM EP Lab,102
2,4,ESJH,ESJH Radiology Special Procedures/Outpatient R...,3030
3,6,EJCH,JC CDU,309
4,5,EUH,EUH Cardiac Cath Lab,265
...,...,...,...,...
85,4,EUH,EUH Cardiac Cath Lab,301
86,6,EUHM,EUHM Cath Lab,196
87,6,EJCH,JC CATH,131
88,5,EUH,EUH Dialysis,507


In [4]:
set(Amb_Patient_Volume.Hospital)

{'EJCH', 'ESJH', 'EUH', 'EUHM'}

## Ambulatory Pt Volume Function

In [5]:
# The variables needed for this function are the: Hospital abbreviation, Hospital NDNQI template, Hospital NDNQI
# code, Calendar year being submitted, and Calendar quarter being submitted
def NDNQI_Amb_Pt_Volume_XML(Selected_Hosp,Hosp_XML_Template,Hosp_NDNQI_Code,Calendar_Year,Calendar_Quarter):
    # This section is to update the Ambulatory Patient Volumes
 
    # The first step takes only the data for this selected hospital
    Hosp_data = Amb_Patient_Volume[Amb_Patient_Volume.Hospital == Selected_Hosp]
    # A List of this Hospital's Unit Names
    Hosp_Units = list(set(Hosp_data.UnitName))

    #This is the current XML template for this hospital and their NDNQI units for this metric
    tree = xml.dom.minidom.parse(Hosp_XML_Template)
    # This is each unit node of the xml file
    unit_nodes = tree.getElementsByTagName("UnitYrQtr")

    # Make sure the correct year and quarter are listed
    H = tree.getElementsByTagName("HospitalCode")[0].childNodes[0].nodeValue = Hosp_NDNQI_Code
    Y = tree.getElementsByTagName("Year")[0].childNodes[0].nodeValue = Calendar_Year
    Q = tree.getElementsByTagName("Quarter")[0].childNodes[0].nodeValue = Calendar_Quarter


    # This loops through each separate UnitName from the Template XML file and replaces it the correct NDNQI
    # name for the units. 
    for idx, unit in enumerate(unit_nodes):
        unit.getElementsByTagName('UnitName')[0].childNodes[0].nodeValue = Hosp_Units[idx]

        # Now grab each of the 3 months for that unit from the Hospital data
        new_data = Hosp_data[Hosp_data.UnitName == Hosp_Units[idx]]
        # Sort the data by month
        new_data = new_data.sort_values('Month')

        # Take each xml Month and replace it with each listed month from the new_data for that unit
        month_nodes = unit.getElementsByTagName("UnitMonth")
        for idx2, month in enumerate(month_nodes):
            month.getElementsByTagName('Month')[0].childNodes[0].nodeValue = new_data.Month.iloc[idx2]


            for column in new_data.columns[3:]:

                new_value = new_data[column].iloc[idx2]
                month.getElementsByTagName(column)[0].childNodes[0].nodeValue = new_value



    # Now, we need to write and save this XML file as its own NDNQI Quarter and Year and Hospital to upload
    # into NDNQI.
    tree.writexml(open('Q{}_{} {} Ambulatory Pt_Volume.xml'.format(Calendar_Quarter, Calendar_Year,Selected_Hosp), 'w'))

# EJCH

In [7]:
NDNQI_Amb_Pt_Volume_XML('EJCH','2020 EJCH Amb Pt Volume Template.xml','YTD069',2021,2)

In [6]:
EJCH_data = Amb_Patient_Volume[Amb_Patient_Volume.Hospital == 'EJCH']
EJCH_data

,Month,Hospital,UnitName,TotalNumberofPatientVisits
3,6,EJCH,JC CDU,309
6,4,EJCH,JCEndo,242
7,5,EJCH,JC CDU,243
15,4,EJCH,JC CATH,119
17,5,EJCH,JCEndo,281
19,5,EJCH,JC CATH,126
29,4,EJCH,JC ROCU,185
31,6,EJCH,JCPain,130
33,5,EJCH,JC ROCU,364
49,4,EJCH,JC CDU,150


## EJSH

In [8]:
NDNQI_Amb_Pt_Volume_XML('ESJH','2020 ESJH Amb Pt Volume Template.xml','XAR317',2021,2)

In [9]:
ESJH_data = Amb_Patient_Volume[Amb_Patient_Volume.Hospital == 'ESJH']
ESJH_data.head()

,Month,Hospital,UnitName,TotalNumberofPatientVisits
2,4,ESJH,ESJH Radiology Special Procedures/Outpatient R...,3030
8,6,ESJH,ESJH Cardiac Rehab/Screening,422
9,6,ESJH,ARU/DRU Procedure Prep/Recovery,1123
10,5,ESJH,Out-patient Infusion/Radiation/Gamma Knife,1586
11,6,ESJH,ESJH Radiology Special Procedures/Outpatient R...,3181


## EUH

In [10]:
NDNQI_Amb_Pt_Volume_XML('EUH','2020 EUH Amb Pt Volume Template.xml','JXE432',2021,2)

In [11]:
EUH_data = Amb_Patient_Volume[Amb_Patient_Volume.Hospital == 'EUH']
EUH_data.head()

,Month,Hospital,UnitName,TotalNumberofPatientVisits
0,4,EUH,EUH EP Lab,273
4,5,EUH,EUH Cardiac Cath Lab,265
14,4,EUH,EUH Endoscopy,444
16,5,EUH,EUH IVR,515
21,4,EUH,EUH IVR,622


## EUHM

In [12]:
NDNQI_Amb_Pt_Volume_XML('EUHM','2020 EUHM Amb Pt Volume Template.xml','PVF155',2021,2)

In [13]:
EUHM_data = Amb_Patient_Volume[Amb_Patient_Volume.Hospital == 'EUHM']
EUHM_data.head()

,Month,Hospital,UnitName,TotalNumberofPatientVisits
1,5,EUHM,EUHM EP Lab,102
5,4,EUHM,EUHM Arrythmia Center,1122
13,4,EUHM,EUHM Endoscopy,490
20,6,EUHM,EUHM Arrythmia Center,1229
24,4,EUHM,EUHM Dialysis,679


## EDH

In [17]:
NDNQI_Amb_Pt_Volume_XML('EDH','2021 EDH Amb Pt Volume Template.xml','NMW346',2021,2)

In [18]:
EDH_data = Amb_Patient_Volume[Amb_Patient_Volume.Hospital == 'EDH']
EDH_data.head()

,Hospital,UnitName,Month,TotalNumberofPatientVisits
3,EDH,EDH EP Lab,4,65
4,EDH,EDH EP Lab,5,32
5,EDH,EDH EP Lab,6,43
6,EDH,EDH Cath Lab,5,135
7,EDH,EDH Cath Lab,4,141


## EHH

In [19]:
NDNQI_Amb_Pt_Volume_XML('EHH','2021 EHH Amb Pt Volume Template.xml','WXX116',2021,2)

In [20]:
EHH_data = Amb_Patient_Volume[Amb_Patient_Volume.Hospital == 'EHH']
EHH_data

,Hospital,UnitName,Month,TotalNumberofPatientVisits
0,EHH,EHH Interventional Procedures,4,188
1,EHH,EHH Interventional Procedures,6,177
2,EHH,EHH Interventional Procedures,5,166


## TEC

In [55]:
NDNQI_Amb_Pt_Volume_XML('TEC','2021 TEC Amb Pt Volume Template.xml','NLH637',2021,2)

In [56]:
TEC_data = Amb_Patient_Volume[Amb_Patient_Volume.Hospital == 'TEC']
TEC_data

,Hospital,UnitName,Month,TotalNumberofPatientVisits
0,TEC,Surgical Services: ENT and Facial Plastic Surg...,6,360
2,TEC,Surgical Services: ENT and Facial Plastic Surg...,4,350
4,TEC,Surgical Services: ENT and Facial Plastic Surg...,5,326
6,TEC,Surgical Services: Urology St Josephs,6,767
8,TEC,Surgical Services: Urology St Josephs,4,756
...,...,...,...,...
734,TEC,Heart and Vascular: Vascular Surgery St Josephs,4,2269
736,TEC,Heart and Vascular: Vascular Surgery St Josephs,5,2218
738,TEC,Heart and Vascular: Cardiology Clifton,6,12588
740,TEC,Heart and Vascular: Cardiology Clifton,4,12939
